In [1]:
from pymatgen import Structure, Lattice, PeriodicSite
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [2]:
from pymatgen.io.vasp.inputs import Kpoints, Poscar, Potcar, Incar
from pymatgen.analysis.elasticity.strain import Deformation
Si_BSn = Structure.from_file("my_part_1/Si_BSn/CONTCAR")
print(Si_BSn)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.633885   3.633885   3.633885
angles:  97.925262  97.925262 136.405454
Sites (2)
  #  SP        a     b     c
---  ----  -----  ----  ----
  0  Si     0.25  0.75   0.5
  1  Si    -0     0     -0


In [3]:
POSCAR = Poscar(Si_BSn)
print(POSCAR)

Si2
1.0
-2.385831 2.385831 1.349347
2.385831 -2.385831 1.349347
2.385831 2.385831 -1.349347
Si
2
direct
0.250000 0.750000 0.500000 Si
-0.000000 0.000000 -0.000000 Si



In [4]:
my_incar_params = {"EDIFF" : 0.0004, 
                "EDIFFG" : -0.01, 
                "ENCUT" : 350, 
                "IBRION" : 2, 
                "ISIF" : 2, 
                "ISMEAR" : 0, 
                "SIGMA" : 0.2, 
                "NSW" : 15,
                "NBANDS" : 8,
                "LREAL" : False}

In [5]:
INCAR = Incar.from_dict(my_incar_params)
print(INCAR)

EDIFF = 0.0004
EDIFFG = -0.01
ENCUT = 350
IBRION = 2
ISIF = 2
ISMEAR = 0
LREAL = False
NBANDS = 8
NSW = 15
SIGMA = 0.2



In [6]:
# fill in this tuple with desired grid dimensions (e.g. (a, b, c))
grid = (8, 8, 8)
# We're using the monkhorst_automatic constructor here to create a grid via the Monkhorst-Pack method.
KPOINTS = Kpoints.monkhorst_automatic(kpts=grid)
print(KPOINTS)

Automatic kpoint scheme
0
Monkhorst
8 8 8



In [7]:
#Define our strain tensor. In this case, e_11 = e_22 = e_33 = 1.10.
strain_tensor = ((1.10, 0, 0), (0, 1.10, 0), (0, 0, 1.10))

deformation = Deformation(strain_tensor)

In [8]:
deformed_Si_BSn = deformation.apply_to_structure(Si_BSn)
print(deformed_Si_BSn)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.997273   3.997273   3.997273
angles:  97.925262  97.925262 136.405454
Sites (2)
  #  SP        a     b     c
---  ----  -----  ----  ----
  0  Si     0.25  0.75   0.5
  1  Si    -0     0     -0


In [9]:
POSCAR = Poscar(deformed_Si_BSn)
print(POSCAR)

Si2
1.0
-2.624414 2.624414 1.484282
2.624414 -2.624414 1.484282
2.624414 2.624414 -1.484282
Si
2
direct
0.250000 0.750000 0.500000 Si
-0.000000 0.000000 -0.000000 Si



In [10]:
def write_input_set(POTCAR, POSCAR, INCAR, KPOINTS, directory="new_input_set", use_fake_potcar=True):
    
    import os
    if not os.path.exists(directory):
        os.makedirs(directory)

    if not use_fake_potcar:
        POTCAR.write_file("{}/POTCAR".format(directory))
    else:
        from shutil import copyfile
        # Copy over a fake potcar
        copyfile("fake_vasp_data/3kvi0H85jC/POTCAR", "directory")
    
    POSCAR.write_file("{}/POSCAR".format(directory))
    INCAR.write_file("{}/INCAR".format(directory))
    KPOINTS.write_file("{}/KPOINTS".format(directory))
    print("Input set written to '{}'".format(directory))

In [11]:
my_directory = "my_part_2/Si_BSn"
write_input_set(None, POSCAR, INCAR, KPOINTS, directory=my_directory)

Input set written to 'my_part_2/Si_BSn'


In [12]:
!python fake_vasp.py "my_part_2/Si_BSn"


No matching pre-computed data for the input files in directory 'my_part_2/Si_BSn' were found:
    Matching POSCAR found? False
    Matching POSCAR + KPOINTS found? Unknown
    Matching POSCAR + KPOINTS + INCAR found? Unknown
Please check your input files and try again.



In [13]:
from pymatgen.io.vasp.outputs import Outcar
OUTCAR = Outcar(my_directory + "/OUTCAR")
OUTCAR.final_energy

-10.27502587